In [2]:
from collections import Counter
from datetime import datetime
 
import json
 
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np

t1 = datetime.now()
with open("yelp_data/dataset/review.json") as f:
    reviews = f.read().strip().split("\n")
reviews = [json.loads(review) for review in reviews]
print(datetime.now() - t1)

0:02:02.268750


In [3]:
texts = [review['text'] for review in reviews]

binstars = [0 if review['stars'] <= 3 else 1 for review in reviews]
bin_texts = []
bin_labels = []
limit = 100000  # Change this to grow/shrink the dataset
neg_pos_counts = [0, 0]
for i in range(len(texts)):
    polarity = binstars[i]
    if neg_pos_counts[polarity] < limit:
        bin_texts.append(texts[i])
        bin_labels.append(binstars[i])
        neg_pos_counts[polarity] += 1

In [4]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(bin_texts)
sequences = tokenizer.texts_to_sequences(bin_texts)
data = pad_sequences(sequences, maxlen=300)

In [5]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'],)
print(len(bin_labels))

200000


In [ ]:
history=model.fit(data, np.array(bin_labels), validation_split=0.5, epochs=10)

Train on 100000 samples, validate on 100000 samples
Epoch 1/10
 30432/100000 [========>.....................] - ETA: 27:45 - loss: 0.3697 - acc: 0.8463